# Network propagation development

The notebook currently covers how results from a AnnData/MuData object can be added to a cpr_graph to setup network-based inference. The general strategy is to:
1. pull out a pd.DataFrame containing feature-level measures of interest along with feature metadata.
2. These are then mapped on the species ids in an sbml_dfs model by shared on ontology, disambiguated (to handle mapping of multiple features to the same s_id), and s_id-indexed results are embedded in the sbml_dfs as a table in species_data
3. attributes of interrest are then passed from the sbml_dfs model into the graph.

This example uses real MuData results but only a small sbml_dfs object which has uniprot but not ENSG identifiers. This makes things easy to work with but a genome-scale graph will need to be used for a real analysis.

Reflecting on the current functionality,

(1) is not too hard but the interface can probably be cleaned up as we should have a function which applies 1-3 in a single call.
(2) is in pretty good shape following a LOT of new functionality being added to napistu-py for handling many-to-one mappings and wide/nested formats for identifiers.
(3) will need some better functionality since the reaction_attrs syntax is pretty cryptic but the core functionality is all there.

Next, steps will be develop basic PPR functionality.

In [1]:
import os

import pandas as pd

from napistu.ingestion import sbml
from napistu import sbml_dfs_core
from napistu import mechanism_matching

# local utils (these should be refactored and removed elsewhere (like in napistu-py))
import utils
import test_utils

# setup logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# paths
PROJECT_DIR =  os.path.expanduser("~/Desktop/Forny_2023_data")
SUPPLEMENTAL_DATA_DIR = os.path.join(PROJECT_DIR, "input")
CACHE_DIR = os.path.join(PROJECT_DIR, "cache")

# Define the path to save hyperparameter scan results
MOFA_PARAM_SCAN_MODELS_PATH = os.path.join(CACHE_DIR, "mofa_param_scan_h5mu")
# Final results 
OPTIMAL_MODEL_H5MU_PATH = os.path.join(CACHE_DIR, "mofa_optimal_model.h5mu")

DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [2]:
PATH_TO_TEST_DATA = os.path.expanduser("~/Desktop/GITHUB/napistu/lib/napistu-py/src/tests/test_data")
example_pathway = os.path.join(PATH_TO_TEST_DATA, "reactome_glucose_metabolism.sbml")
assert os.path.exists(example_pathway)

In [3]:
sbml_dfs = sbml_dfs_core.SBML_dfs(sbml.SBML(example_pathway))

species_identifiers = sbml_dfs.get_identifiers("species").query("bqb == 'BQB_IS'").query("ontology != 'reactome'")

INFO:napistu.utils:creating an edgelist linking index levels s_id, entry and linking it to levels defined by ontology, identifier
DEBUG:napistu.utils:label is not defined in table_schema; adding a constant (1)


In [4]:
# lets load the Forny results so we trying adding a few different types of tables to the sbml_dfs

import mudata as md
mdata = md.read_h5mu(OPTIMAL_MODEL_H5MU_PATH)


DEBUG:h5py._conv:Creating converter from 3 to 5
/Users/maya/Desktop/GITHUB/napistu/lib/napistu-scrapyard/applications/forny_2023/.venv/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/maya/Desktop/GITHUB/napistu/lib/napistu-scrapyard/applications/forny_2023/.venv/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_a

In [5]:
# results from var
var_level_results = mdata["proteomics"].var[["effect_case", "qval_case"]].copy()
var_level_results.index.name = "feature_id"
var_level_results['uniprot'] = var_level_results.index.to_series()

mechanism_matching.bind_wide_results(
    sbml_dfs,
    var_level_results,
    "var_level_results",
    ontologies = {"uniprot"},
    dogmatic = False,
    verbose = True
)

sbml_dfs.species_data["var_level_results"]

INFO:napistu.sbml_dfs_utils:Running in non-dogmatic mode - genes, transcripts, and proteins will be merged if possible.
DEBUG:napistu.mechanism_matching:Validated ontology columns: {'uniprot'}
INFO:napistu.mechanism_matching:Using columns as results: ['qval_case', 'effect_case', 'feature_id']
DEBUG:napistu.mechanism_matching:Final long format shape: (4788, 5)
DEBUG:napistu.mechanism_matching:Matching 4788 features to 98 species for ontology uniprot
INFO:napistu.mechanism_matching:Found 57 total matches across 1 ontologies
INFO:napistu.mechanism_matching:1.3% of feature_ids are present one or more times in the output (57/4513)
INFO:napistu.mechanism_matching:7 s_id(s) map to more than one feature_id.
INFO:napistu.mechanism_matching:Examples of s_id mapping to multiple feature_ids (showing up to 3):
s_id       s_name           
S00000016  SLC25A12,13               [541, 4488]
S00000031  enolase dimer        [833, 906, 1034]
S00000050  aldolase tetramer          [773, 934]
Name: feature_i

,effect_case,qval_case,feature_id
s_id,,,
S00000006,0.064783,0.996212,1512
S00000012,0.059317,0.996212,4425
S00000013,0.290775,0.996212,2156
S00000015,0.079570,0.996212,1513
S00000016,0.021029,0.996212,"4488,541"
S00000019,0.047774,0.996212,1767
S00000022,0.202818,0.996212,984
S00000031,0.165435,0.996660,"1034,833,906"
S00000033,0.083939,0.996212,730


In [6]:
# merge factors with metadata
mofa_dfs_dict = utils.split_varm_by_modality(mdata)

modality = "transcriptomics"

mofa_df_list = list()
for modality in mofa_dfs_dict.keys():

    modality_pk = mofa_dfs_dict[modality].index.name
    filter_col = [col for col in mofa_dfs_dict[modality] if col.startswith('LF')]
    modality_df = mofa_dfs_dict[modality][filter_col].copy()
    modality_df.index.name = "feature_id"
    modality_df[modality_pk] = modality_df.index.to_series()
    modality_df["modality"] = modality

    mofa_df_list.append(modality_df)

mofa_df = pd.concat(mofa_df_list, axis=0)

mofa_df.groupby("modality").sample(5)


,LFs1,LFs2,LFs3,LFs4,LFs5,LFs6,LFs7,LFs8,LFs9,LFs10,...,LFs24,LFs25,LFs26,LFs27,LFs28,LFs29,LFs30,ensembl_gene,modality,uniprot
feature_id,,,,,,,,,,,,,,,,,,,,,
Q96EC8,-0.078180,-0.003538,0.877200,-0.005746,0.002101,-0.027462,-0.064789,0.145216,-0.075275,0.540646,...,0.019828,0.003780,-0.003639,-0.045765,0.000964,-0.002235,-0.001720,NaN,proteomics,Q96EC8
Q16540,-0.005176,0.064878,-0.005516,-0.000463,-0.016086,-0.001495,0.107386,0.181010,-0.032700,-0.024537,...,-0.006981,0.001210,-0.001954,-0.037268,0.003657,-0.003429,-0.000725,NaN,proteomics,Q16540
P60983,0.001681,0.042997,0.016273,0.004630,0.001098,0.003500,0.000891,-0.351672,0.030620,-0.085256,...,-0.001077,-0.000695,-0.002018,0.034438,0.010717,-0.005510,0.000644,NaN,proteomics,P60983
Q9UI30,-0.046268,0.061522,0.050759,0.000007,0.001951,0.027214,0.010494,-0.067158,-0.019496,0.010423,...,-0.024191,-0.003317,-0.001556,0.136762,0.001863,-0.002126,0.003767,NaN,proteomics,Q9UI30
Q9NQG5,0.046269,0.003717,-0.065278,-0.003860,0.006038,-0.002320,0.003738,0.089707,-0.009675,-0.012850,...,0.024639,0.000007,-0.001938,-0.028932,-0.007981,0.081167,-0.001806,NaN,proteomics,Q9NQG5
ENSG00000107566,-0.004875,0.094709,0.000218,0.028670,0.022667,-0.024981,-0.050551,-0.006511,0.005133,0.000044,...,-0.012236,-0.004441,0.103528,0.014399,0.132553,0.153140,0.091975,ENSG00000107566,transcriptomics,NaN
ENSG00000185803,-0.033563,-0.013176,0.000076,-0.100077,-0.020662,0.123703,-0.038562,0.001762,0.049872,0.000189,...,0.190878,-0.242584,0.164857,-0.245696,0.235013,0.092763,0.018315,ENSG00000185803,transcriptomics,NaN
ENSG00000131473,-0.149243,0.000032,0.000021,0.030772,-0.064339,0.049610,-0.024688,0.000494,0.093107,-0.000022,...,-0.014739,0.069637,-0.046225,-0.144591,-0.030901,0.075353,0.020989,ENSG00000131473,transcriptomics,NaN
ENSG00000137310,-0.240772,0.235774,0.000089,-0.054507,0.012061,0.036914,0.067204,-0.002622,0.149849,-0.000039,...,0.132113,-0.235888,-0.064105,0.293744,0.027725,0.033537,-0.123580,ENSG00000137310,transcriptomics,NaN


In [7]:
mechanism_matching.bind_wide_results(
    sbml_dfs,
    mofa_df,
    "mudata_varm_results",
    ontologies = {"uniprot", "ensembl_gene"},
    dogmatic = False,
    verbose = True
)

sbml_dfs.species_data["mudata_varm_results"]

INFO:napistu.sbml_dfs_utils:Running in non-dogmatic mode - genes, transcripts, and proteins will be merged if possible.
DEBUG:napistu.mechanism_matching:Validated ontology columns: {'uniprot', 'ensembl_gene'}
INFO:napistu.mechanism_matching:Using columns as results: ['LFs22', 'LFs1', 'LFs6', 'LFs30', 'LFs19', 'LFs27', 'LFs18', 'LFs10', 'LFs13', 'LFs8', 'LFs20', 'LFs4', 'LFs11', 'LFs15', 'LFs5', 'LFs29', 'LFs7', 'feature_id', 'LFs23', 'LFs16', 'LFs2', 'LFs24', 'LFs3', 'modality', 'LFs12', 'LFs25', 'LFs14', 'LFs21', 'LFs28', 'LFs26', 'LFs17', 'LFs9']
DEBUG:napistu.mechanism_matching:Final long format shape: (13922, 34)
DEBUG:napistu.mechanism_matching:Matching 4788 features to 98 species for ontology uniprot
INFO:napistu.mechanism_matching:Found 57 total matches across 1 ontologies
INFO:napistu.mechanism_matching:0.4% of feature_ids are present one or more times in the output (57/13647)
INFO:napistu.mechanism_matching:7 s_id(s) map to more than one feature_id.
INFO:napistu.mechanism_matc

,LFs1,LFs2,LFs3,LFs4,LFs5,LFs6,LFs7,LFs8,LFs9,LFs10,...,LFs23,LFs24,LFs25,LFs26,LFs27,LFs28,LFs29,LFs30,modality,feature_id
s_id,,,,,,,,,,,,,,,,,,,,,
S00000006,0.051530,0.036362,-0.041906,-0.001347,-0.013508,0.001565,-0.021276,-0.244355,-0.031964,0.057272,...,-0.191047,0.075975,-0.006429,0.009816,0.074251,0.010391,0.000629,-0.004023,proteomics,10646
S00000012,0.034465,0.115698,-0.047133,-0.001525,-0.006179,0.015768,0.042666,0.118975,0.002245,0.035891,...,0.153040,-0.007040,0.000832,0.005247,0.003451,0.000202,0.001349,-0.002890,proteomics,13559
S00000013,0.111319,-0.002571,-0.073818,-0.000615,-0.066941,0.040771,-0.002893,0.120330,-0.024400,0.337955,...,-0.075491,-0.005003,0.001831,-0.016633,-0.083339,-0.000686,0.003954,-0.002430,proteomics,11290
S00000015,0.010020,0.043131,-0.000815,0.000010,-0.032613,0.026158,0.044396,0.171346,0.001049,0.003718,...,-0.071206,0.054874,-0.052803,-0.000263,-0.014903,0.000827,0.036917,-0.002143,proteomics,10647
S00000016,0.099801,0.064125,-0.015871,-0.001283,-0.085869,0.002985,0.009716,0.216461,-0.005344,0.003299,...,-0.044812,0.032438,-0.001762,0.001995,-0.094416,-0.005864,-0.006293,-0.005239,proteomics,"13622,9675"
S00000019,0.064609,0.006714,-0.110616,-0.017464,-0.020428,-0.032641,-0.003856,0.083932,0.000541,-0.054995,...,-0.121061,-0.032175,-0.006698,-0.002780,0.095618,0.002672,0.074969,-0.004676,proteomics,10901
S00000022,0.034890,0.007629,-0.122095,-0.001708,-0.076321,0.061220,-0.055165,-0.110623,0.014309,0.324787,...,-0.065491,-0.016958,-0.011363,-0.004861,-0.308404,0.006624,-0.031057,0.000834,proteomics,10118
S00000031,0.014932,-0.100028,-0.056794,-0.015458,-0.039523,0.024284,-0.102462,-0.278852,-0.129555,0.008516,...,-0.225068,0.005913,-0.008280,-0.010984,0.235486,0.014511,0.010286,0.005220,proteomics,"10040,10168,9967"
S00000033,0.055625,-0.149947,0.003432,-0.031034,-0.020317,0.054525,-0.158043,-0.283876,-0.124461,0.013506,...,-0.285335,0.016808,-0.004187,-0.003736,0.322549,0.018490,0.048649,-0.000388,proteomics,9864


In [11]:
from napistu.network import net_create

# now we can pass these species_data attributes to the graph

reaction_graph_attrs = {
    "species": {
        "LFs5": {
            "table": "mudata_varm_results",
            "variable": "LFs5",
            "trans": "abs",
        },
        "effect_case": {
            "table": "var_level_results",
            "variable": "effect_case",
            "trans": "abs",
        },
    },
}

cpr_graph = net_create.create_cpr_graph(
    sbml_dfs,
    directed=True,
    graph_type="regulatory"
)

# add species attributes
# TO DO - this is definitely not a utility function
graph_w_annotations = net_create._add_graph_species_attribute(
    cpr_graph,
    sbml_dfs,
    species_graph_attrs = reaction_graph_attrs,
    custom_transformations = {
        # take the absolute value
        "abs" : lambda x: abs(x)
    }
)


INFO:napistu.network.net_create:Organizing all network nodes (compartmentalized species and reactions)


INFO:napistu.network.net_create:Formatting edges as a regulatory graph
INFO:napistu.network.net_create:Formatting 250 reactions species as tiered edges.
INFO:napistu.network.net_create:Adding additional attributes to edges, e.g., # of children and parents.
INFO:napistu.network.net_create:Done preparing regulatory graph
INFO:napistu.network.net_create:Adding reversibility and other meta-data from reactions_data
INFO:napistu.network.net_create:No reactions annotations provided in "graph_attrs"; returning None
INFO:napistu.network.net_create:Creating reverse reactions for reversible reactions on a directed graph
INFO:napistu.network.net_create:Formatting cpr_graph output
INFO:napistu.network.net_create:Adding meta-data from species_data
INFO:napistu.network.net_create:Adding new attribute LFs5 to vertices
INFO:napistu.network.net_create:Adding new attribute effect_case to vertices


In [13]:
from napistu import utils as napistu_utils

napistu_utils.style_df(graph_w_annotations.get_vertex_dataframe().sort_values("LFs5").head(5))

,name,node_name,node_type,LFs5,effect_case
vertex ID,,,,,
0,species_113780,Glc [endoplasmic reticulum lumen],species,0.000,0.000
101,species_76116,Glycerol [cytosol],species,0.000,0.000
103,species_163745,phosphoPFKFB1 dimer [cytosol],species,0.000,0.000
104,species_71786,PFKFB1 dimer [cytosol],species,0.000,0.000
107,reaction_198458,Efflux of glucose from the endoplasmic reticulum,reaction,0.000,0.000


## Network Propagation

Here we'll implement a workflow for applying network propagation to a cpr_graph's vertex attributes.

In [14]:
test_utils.test_personalized_pagerank_by_attribute_basic()
test_utils.test_personalized_pagerank_by_attribute_no_uniform()
test_utils.test_personalized_pagerank_by_attribute_missing_and_negative()
test_utils.test_personalized_pagerank_by_attribute_additional_args_invalid()
test_utils.test_personalized_pagerank_by_attribute_all_missing()
test_utils.test_personalized_pagerank_by_attribute_all_zero()


In [15]:
graph_w_annotations.vs.attributes()

['name', 'node_name', 'node_type', 'LFs5', 'effect_case']

In [16]:
RESET_PROPORTIONAL_TO = "effect_case"

utils.personalized_pagerank_by_attribute(
    graph_w_annotations,
    RESET_PROPORTIONAL_TO
)

,name,pagerank_by_attribute,effect_case,pagerank_uniform
0,species_113780,0.001163,0.0,0.000953
1,species_70115,0.000840,0.0,0.000688
2,species_981603,0.000000,0.0,0.000000
3,species_113782,0.003220,0.0,0.002637
4,species_113519,0.000000,0.0,0.000000
...,...,...,...,...
152,reaction_8955794,0.007420,0.0,0.006999
153,reaction_163750,0.002946,0.0,0.003794
154,reaction_70262,0.003240,0.0,0.003417
155,reaction_71802,0.006678,0.0,0.006950
